### Bước 1: Cài đặt các thư viện cần thiết

In [1]:
!pip install tensorflow
!pip install split-folders

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import splitfolders

### Bước 2: Đặt đường dẫn home chứa folder project hiện tại

In [2]:
import platform

if platform.system() == 'Linux':
    home_dir = '/content/drive/MyDrive/ClassificationofMangoDiseases'
elif platform.system() == 'Windows':
    home_dir = 'D:\Projects\ClassificationofMangoDiseases'
else:
    raise ValueError('Unsupported platform')

print(f'Home directory: {home_dir}')

### Bước 3: Kết nối với Google Drive nếu đang chạy trên Google Colab

In [3]:
if platform.system() == 'Linux':
    from google.colab import drive
    drive.mount('/content/drive')

### Bước 4: Đặt đường dẫn tới tập dữ liệu

In [4]:
# Đường dẫn tới tập dữ liệu đã chọn (gốc hoặc đã remove background)
data_dir = os.path.join(home_dir, 'data/MangoFruitDDS/SenMangoFruitDDS_bgremoved')  # Hoặc 'SenMangoFruitDDS_original'
output_dir = os.path.join(home_dir, 'data/processed')

print(f'Data directory: {data_dir}')
print(f'Output directory: {output_dir}')

### Bước 5: Tiền xử lý dữ liệu và phân chia tập dữ liệu

Sử dụng thư viện `split-folders` để phân chia tập dữ liệu thành các tập `train`, `valid`, và `test`.

In [5]:
# Xóa thư mục đầu ra nếu đã tồn tại
if os.path.exists(output_dir):
    import shutil
    shutil.rmtree(output_dir)

# Phân chia dữ liệu
splitfolders.ratio(data_dir, output=output_dir, seed=42, ratio=(.7, .2, .1), group_prefix=None)

### Bước 6: Hiển thị thông tin thư mục sau khi phân chia

In [6]:
# Kiểm tra các thư mục sau khi phân chia
for root, dirs, files in os.walk(output_dir):
    level = root.replace(output_dir, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print('{}{}/'.format(indent, os.path.basename(root)))
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print('{}{}'.format(subindent, f))

### Bước 7: Xác nhận kết quả

Kiểm tra lại thư mục đã được phân chia đúng cách.

In [7]:
import matplotlib.pyplot as plt
import random
from PIL import Image

# Hiển thị một số hình ảnh từ tập train
train_dir = os.path.join(output_dir, 'train')
class_names = os.listdir(train_dir)
print(f'Classes: {class_names}')

# Chọn ngẫu nhiên một lớp
random_class = random.choice(class_names)
print(f'Random class: {random_class}')

# Chọn ngẫu nhiên một hình ảnh từ lớp đó
random_image_path = os.path.join(train_dir, random_class, random.choice(os.listdir(os.path.join(train_dir, random_class))))
print(f'Random image path: {random_image_path}')

# Hiển thị hình ảnh
image = Image.open(random_image_path)
plt.imshow(image)
plt.axis('off')
plt.show()

### Thiết lập siêu tham số và chức năng chung cho các mô hình

In [8]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
import random
import os

# Set random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

# Define common hyperparameters
batch_size = 32
epochs = 100
learning_rate = 0.001

# Define common loss function and optimizer
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Define image size and input shape
img_height = 224
img_width = 224
input_shape = (img_height, img_width, 3)

### Hàm tạo các mô hình

In [9]:
# Function to create a simple CNN model
def create_cnn_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Function to create a ResNet model
def create_resnet_model(input_shape, num_classes):
    base_model = tf.keras.applications.ResNet50(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Function to create an Inception model
def create_inception_model(input_shape, num_classes):
    base_model = tf.keras.applications.InceptionV3(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Function to create a DenseNet model
def create_densenet_model(input_shape, num_classes):
    base_model = tf.keras.applications.DenseNet121(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Function to create a model using Transfer Learning
def create_transfer_learning_model(input_shape, num_classes):
    base_model = tf.keras.applications.VGG16(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Example usage:
num_classes = 5  # Adjust this according to your dataset
cnn_model = create_cnn_model(input_shape, num_classes)
resnet_model = create_resnet_model(input_shape, num_classes)
inception_model = create_inception_model(input_shape, num_classes)
densenet_model = create_densenet_model(input_shape, num_classes)
transfer_learning_model = create_transfer_learning_model(input_shape, num_classes)

# Compile the models
models = [cnn_model, resnet_model, inception_model, densenet_model, transfer_learning_model]
for model in models:
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

# Now you can use these models for training on your dataset

### Huấn luyện và lưu các mô hình

In [10]:
# Define directories to save models
if platform.system() == 'Linux':
    model_save_dir = '/content/drive/MyDrive/ClassificationofMangoDiseases/models'
elif platform.system() == 'Windows':
    model_save_dir = 'D:\Projects\ClassificationofMangoDiseases\models'
else:
    raise ValueError('Unsupported platform')

os.makedirs(model_save_dir, exist_ok=True)

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(output_dir, 'train'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'
)

validation_generator = test_datagen.flow_from_directory(
    os.path.join(output_dir, 'val'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'
)

# Train and save each model
for model, name in zip(models, ['cnn', 'resnet', 'inception', 'densenet', 'transfer_learning']):
    print(f'Training {name} model...')
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator
    )
    model_path = os.path.join(model_save_dir, f'{name}_model.h5')
    model.save(model_path)
    print(f'{name} model saved at {model_path}')